In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import torch 
from argparse import Namespace
from tqdm import tqdm
import pickle 
import glob 
import ast

# Add the path to the directory containing the sybil module
sys.path.append('/workspace/home/tengyuezhang/sybil_cect/code/Sybil/')
from sybil.utils.metrics import concordance_index, get_survival_metrics
from sybil import Sybil, Serie

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
num_threads = os.cpu_count() // 5

In [3]:
# Initialize the Sybil model
model = Sybil("sybil_ensemble")
num_years = 6

/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
df = pd.DataFrame(columns=['directory', 'pid', 'event'])

# Data to add
row1 = {
    'directory': '/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714',
    'pid': 'LUNGX-CT001',
    'event': 1
}

row2 = {
   'pid': 'LUNGX-CT002',
    'directory': '/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT002/05-25-2007-914-CT INFUSED CHEST-96111/4.000000-HIGH RES-70218', 
    'event': 0
}

# Add the new row to the DataFrame using pandas concat
row1_df = pd.DataFrame([row1])
row2_df = pd.DataFrame([row2])
df = pd.concat([df, row1_df], ignore_index=True)
df = pd.concat([df, row2_df], ignore_index=True)

df


,directory,pid,event
0,/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-C...,LUNGX-CT001,1
1,/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-C...,LUNGX-CT002,0


In [47]:
for i in range(num_years):
    df[f'pred_risk_year_{i}'] = np.nan


for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing cases"):
    
    dicom_dir = f"/workspace{row['directory']}"
    event = row['event']
    years_to_event = 1
    pid = row['pid']
    dicom_list = glob.glob(dicom_dir + '/*')
    # serie = Serie(dicom_list, label=event, censor_time=years_to_event)
    serie = Serie(dicom_list)
    results = model.predict([serie], return_attentions=True, threads=10)
        
    # Update the risk scores columns for the current row
    for i in range(num_years):
        df.at[index, f'pred_risk_year_{i}'] = results.scores[0][i]

Processing cases: 100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.47s/it]


In [48]:
df

,directory,pid,event,pred_risk_year_0,pred_risk_year_1,pred_risk_year_2,pred_risk_year_3,pred_risk_year_4,pred_risk_year_5
0,/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-C...,LUNGX-CT001,1,0.162194,0.266052,0.257810,0.294426,0.326781,0.382972
1,/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-C...,LUNGX-CT002,0,0.021629,0.038573,0.071919,0.079270,0.095846,0.135681


In [24]:
model = Sybil("sybil_ensemble")
dicom_dir = '/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714'
dicoms = [os.path.join(dicom_dir,i) for i in os.listdir(dicom_dir)]
print(dicoms)
serie = Serie(dicoms)
scores = model.predict ([serie], threads = 10)
risk_scores = scores[0][0]

['/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714/1-022.dcm', '/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714/1-234.dcm', '/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714/1-164.dcm', '/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714/1-046.dcm', '/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714/1-248.dcm', '/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714/1-053.dcm', '/workspace/data/lung/SPIE_AAPM_Lung_CT_Challenge/LUNGx-CT001/09-24-2006-7805-CT INFUSED CHEST-62081/4.000000-HIGH RES-69714/1-127.dcm', '/workspace/data/lung/SPIE_AAPM_Lung_CT_

In [27]:
dicoms1 == dicoms

False

In [22]:
scores

Prediction(scores=[[0.16219448545852416, 0.2660521773838938, 0.2578104393170584, 0.294426092782818, 0.3267814189024213, 0.3829717786744629]], attentions=None)

None
